# Extracting Tables from Websites into .csv Files in Python

*Original source code/method here:* https://stackoverflow.com/questions/10556048/how-to-extract-tables-from-websites-in-python/44506462


A common problem we face with publicly available aggregate data is that it is often presented in tables on webpages, with no way to conveniently download the data. A solution is to write a script that will download the data for you and save it as a .csv file.

Pandas can do this right out of the box, saving you from having to parse the html yourself!

There are two options presented:
1. A script
2. The same code but written in a function

Both ultimately achieve the same thing - reading in the table from the URL and tidying up the data.

## Import Dependencies
- [BeatifulSoup](https://pypi.org/project/beautifulsoup4/) - 
- [lxml](https://lxml.de/) - 
- [html5lib](https://pypi.org/project/html5lib/) - 
- [pandas](https://pandas.pydata.org/) - 
- [requests](https://requests.readthedocs.io/en/master/user/quickstart/) - 

In [ ]:
# OPTIONAL: Install packages, if not installed already
#pip install bs4
#pip install lxml
#pip install html5lib

In [54]:
# Import 3rd party dependencies
import requests
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import html5lib

# Option 1: An example script to read in and wrangle data

For these examples, we will be reading in staff demographic data from the California Department of Education.
https://dq.cde.ca.gov/dataquest/dqcensus/StfFteClassifiedLevels.aspx?cds=00&agglevel=State&year=2019-20

* `read_html()` extracts all tables from your html and puts them in a list of DataFrames.  
* `df_list[-2]` is used in the example to extract the 2nd to last table from the web page.
* `to_csv()` is used to convert each dataframe to a csv file.

In [55]:
# Assign your URL as a string to `url`
url = 'https://dq.cde.ca.gov/dataquest/dqcensus/StfFteClassifiedLevels.aspx?cds=00&agglevel=State&year=2019-20'

# Get the html content with `requests`
html = requests.get(url).content

# Read the html with `read_html()`
df_list = pd.read_html(html)

# Get the last DataFrame
df = df_list[-2]

# Delete the `Total` column
df.drop(columns=['Total'], inplace = True)

# Melt the columns
df_melted = pd.melt(df, id_vars=['Name'], var_name = 'Ethnicity', value_name = 'Number of FTE Staff')

# Rename the first column
df_melted.rename(columns={'Name':'County'}, inplace = True)

# Save the DataFrames to a .csv file
df_melted.to_csv('my data3.csv')

df_melted.head()

,County,Ethnicity,Number of FTE Staff
0,Alameda,"American Indianor Alaska Native,not Hispanic",83.79
1,Alpine,"American Indianor Alaska Native,not Hispanic",1.00
2,Amador,"American Indianor Alaska Native,not Hispanic",5.40
3,Butte,"American Indianor Alaska Native,not Hispanic",30.10
4,Calaveras,"American Indianor Alaska Native,not Hispanic",8.36


# Option 2: Creating a function to extract tables from websites

In [40]:
# Function to read a DataFrame from a webpage
def table_to_df(url, dataframe_pos = -1):
    
    # Inputs:
    # url as a string
    # dataframe_pos as an integer
    
    # Outputs:
    # returns a DataFrame, df_created
    
    # Get the html content with `requests`
    html = requests.get(url).content
    
    # Read the html with `read_html()`
    df_list = pd.read_html(html)
    
    # Get the DataFrame based on dataframe_pos
    df_created = df_list[dataframe_pos]
    
    return df_created

In [47]:
# Use the function to create a DataFrame from the html link.
df_created_new = table_to_df('https://dq.cde.ca.gov/dataquest/dqcensus/StfFteClassifiedLevels.aspx?cds=00&agglevel=State&year=2019-20', -2)

In [48]:
# Examine the data
df_created_new.head()

,Name,"American Indianor Alaska Native,not Hispanic","Asian,not Hispanic","Pacific Islander,not Hispanic","Filipino,not Hispanic",Hispanic,"African American,not Hispanic","White,not Hispanic","Two or More Races,not Hispanic",Total
0,Alameda,83.79,879.68,147.02,276.78,1952.22,909.29,2553.70,1302.02,8104.50
1,Alpine,1.00,0.00,0.00,0.00,1.00,0.00,16.75,0.00,18.75
2,Amador,5.40,0.00,1.00,0.37,7.72,1.00,174.90,2.69,193.08
3,Butte,30.10,63.89,6.56,6.57,204.49,45.70,1238.25,41.27,1636.83
4,Calaveras,8.36,3.59,2.34,2.71,25.83,2.44,282.26,5.00,332.53


Our DataFrame is created and is now ready for wrangling.

In [52]:
# Delete the `Total` column
df_created_new.drop(columns=['Total'], inplace = True)

# Melt and rename the columns
df_created_new = pd.melt(df_created_new, id_vars=['Name'], var_name = 'Ethnicity', value_name = 'Number of FTE Staff')
df_created_new.rename(columns={'Name':'County'}, inplace = True)

In [53]:
# Save the DataFrames to a .csv file
df_created_new.to_csv('my data4.csv')

# Examine the data
df_created_new.head()

,County,Ethnicity,Number of FTE Staff
0,Alameda,"American Indianor Alaska Native,not Hispanic",83.79
1,Alpine,"American Indianor Alaska Native,not Hispanic",1.00
2,Amador,"American Indianor Alaska Native,not Hispanic",5.40
3,Butte,"American Indianor Alaska Native,not Hispanic",30.10
4,Calaveras,"American Indianor Alaska Native,not Hispanic",8.36
